In [ ]:
%%capture
# Core libraries for agentic workflows and LangGraph
!pip install "langgraph==0.6.6" "langchain==0.3.7" "langchain-community==0.3.7"

# Embeddings / models / vector search (all free/open-source friendly)
!pip install "sentence-transformers==3.1.1" "faiss-cpu==1.8.0.post1"

# For visualizing workflows/graphs
!apt-get update
!apt-get install -y graphviz libgraphviz-dev
!pip install "pygraphviz==1.14"

In [ ]:
# Importing the core pieces of LangGraph that I'll need to build the workflow
from langgraph.graph import StateGraph, START, END

# I'll use TypedDict to define the structured state that flows through my graph
from typing import TypedDict

# Using Pydantic models to keep inputs/outputs clean and validated
from pydantic import BaseModel, Field

# Keeping things provider-agnostic for now; I'll plug in the actual LLM later
# (either a small Hugging Face model or a free local backend in Colab)
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import HumanMessage, AIMessage


In [ ]:
# I’m adding a small helper function so I can quickly inspect any workflow I build.
# This makes it easier for me to debug the graph structure as it gets more complex.

def print_workflow_info(workflow, app=None):
    """Utility to print out the key details of a LangGraph workflow so I can understand the structure at a glance."""

    print("WORKFLOW INFORMATION")
    print("====================")

    # Basic overview of the graph
    print(f"Nodes: {workflow.nodes}")
    print(f"Edges: {workflow.edges}")

    # Checking how the workflow defines its finish points
    try:
        finish_points = workflow.finish_points
        print(f"Finish points: {finish_points}")
    except:
        try:
            # Some versions expose the attribute differently, so I’m covering that too
            print(f"Finish point: {workflow._finish_point}")
        except:
            print("Finish points attribute not directly accessible")

    # If I'm passing the compiled app, I can also visualize the graph
    if app:
        print("\nWorkflow Visualization:")
        from IPython.display import display
        display(app.get_graph().draw_png())


In [ ]:
# For this project I don’t want to rely on paid APIs, so I’m loading a small open-source chat model instead.
# This keeps the notebook fully runnable in Colab.

from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# I’m using a tiny model so it runs comfortably on Colab’s free GPU/CPU.
model_id = "HuggingFaceH4/zephyr-7b-beta"

# Loading tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",              # lets Colab decide CPU/GPU placement
    torch_dtype="auto"              # keeps memory usage manageable
)

# Wrapping the model into a LangChain-compatible pipeline
chat_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256
)

# This is the LLM object I'll use throughout the workflow
llm = HuggingFacePipeline(pipeline=chat_pipeline)


In [ ]:
# I’m defining the state that will move through my LangGraph workflow.
# Keeping it typed helps me stay organized as the graph grows.

class ChainState(TypedDict):
    job_description: str
    resume_summary: str
    cover_letter: str


In [ ]:
# This function generates a resume summary based on the job description.
# I’m keeping the prompt simple and focused so it works well with open-source models too.

def generate_resume_summary(state: ChainState) -> ChainState:
    prompt = f"""
You are a resume assistant. Read the job description below and produce a short,
strong resume-style summary that reflects what an ideal applicant would highlight.

Job Description:
{state['job_description']}
"""

    # For HuggingFacePipeline models, calling the LLM directly returns a string,
    # so I don’t need to access `.content` like with OpenAI models.
    response = llm(prompt)

    return {**state, "resume_summary": response}


In [ ]:
# This node generates a cover letter based on the resume summary and job description.
# I’m keeping the prompt clear so the model has enough structure to produce a good output.

def generate_cover_letter(state: ChainState) -> ChainState:
    prompt = f"""
You are a cover-letter assistant. Using the resume summary below, write a
professional, confident, and personalized cover letter for the job.

Resume Summary:
{state['resume_summary']}

Job Description:
{state['job_description']}
"""

    # HuggingFacePipeline returns plain text, so I just call it directly.
    response = llm(prompt)

    return {**state, "cover_letter": response}


In [ ]:
# Now I’m setting up the LangGraph workflow and telling it
# what kind of state it will pass around between nodes.

workflow = StateGraph(ChainState)

# Just returning it here so I can quickly inspect the object in the notebook.
workflow


In [ ]:
# Adding my two main nodes to the workflow. Each node handles one step
# in the overall agentic process I'm building.

workflow.add_node("generate_resume_summary", generate_resume_summary)
workflow.add_node("generate_cover_letter", generate_cover_letter)


In [ ]:
# Setting the first step of my workflow. The graph will always start
# by generating the resume summary before moving on to anything else.

workflow.set_entry_point("generate_resume_summary")


In [ ]:
# Once the resume summary is generated, I want the workflow to
# automatically move to the cover-letter step, so I'm connecting the nodes.

workflow.add_edge("generate_resume_summary", "generate_cover_letter")


In [ ]:
# I'm telling the graph where it should stop. Once the cover letter is generated,
# the workflow has everything it needs, so that becomes my finish point.

workflow.set_finish_point("generate_cover_letter")

# Quick check to make sure the graph looks the way I expect.
print_workflow_info(workflow)


In [ ]:
# Compiling the workflow so it becomes a runnable LangGraph app.
# This turns the structure I defined above into an executable pipeline.

app = workflow.compile()


In [ ]:
from IPython.display import Image, display

# Just visualizing the workflow here so I can see the nodes and edges clearly.
# This helps me double-check that the structure matches what I intended.

display(Image(app.get_graph().draw_png()))


In [ ]:
# Here I'm preparing a simple test input so I can run the full workflow.
# This helps me confirm that both nodes (resume + cover letter) are working correctly.

input_state = {
    "job_description": (
        "We are looking for a data scientist with experience in machine learning, NLP, and Python. "
        "Prior work with large datasets and experience deploying models into production is required."
    )
}

# Running the graph with my input to see if everything connects the right way.
result = app.invoke(input_state)

# Checking the generated resume summary
result['resume_summary']


In [ ]:
# Defining a new state type for the routing workflow.
# This one will let me classify what the user wants and route it to the right step.

class RouterState(TypedDict):
    user_input: str
    task_type: str
    output: str


In [ ]:
# I’m defining a simple Router schema so I know exactly what the model
# is supposed to output whenever I ask it to classify a task.

class Router(BaseModel):
    role: str = Field(
        ...,
        description=(
            "Decide whether the user wants to summarize a passage (output 'summarize') "
            "or translate text into French (output 'translate')."
        )
    )

# Since HuggingFace models don’t have built-in tool-calling like OpenAI,
# I’m writing a small helper that prompts the model and forces a clean decision.

def route_task(user_input: str) -> str:
    prompt = f"""
You are a routing assistant. Read the user input and decide the task type.

If the user wants a summary → respond ONLY with: summarize
If the user wants French translation → respond ONLY with: translate

User Input:
{user_input}

Respond with one word only.
"""
    # Getting the raw text output from my HF model
    raw_output = llm(prompt).strip().lower()

    # Cleaning the output so it fits my Router model
    if "summar" in raw_output:
        return "summarize"
    if "translat" in raw_output or "french" in raw_output:
        return "translate"

    # Fallback in case the model is indecisive
    return "summarize"

# Quick test
response = route_task("summarize this: I love the sun, it's so warm")
response


In [ ]:
# This node is my classifier. It looks at the user's input and decides
# what kind of task they’re asking for, so I can route it to the right branch.

def router_node(state: RouterState) -> RouterState:
    routing_prompt = f"""
You are an AI task classifier.

Decide whether the user wants to:
- "summarize" a passage
- or "translate" text into French

Respond with just one word: summarize or translate.

User Input:
{state['user_input']}
"""

    # Reusing the helper I defined earlier so that the routing logic stays in one place.
    task_type = route_task(state["user_input"])

    # I store the decision in 'task_type' so the next node can use it for routing.
    return {**state, "task_type": task_type}


In [ ]:
# This small helper just tells LangGraph which branch to follow next
# based on the decision I stored in the state.

def router(state: RouterState) -> str:
    return state["task_type"]


In [ ]:
# This node handles the "summarize" path.
# It takes the user input and asks the model for a short summary.

def summarize_node(state: RouterState) -> RouterState:
    prompt = f"Please summarize the following passage:\n\n{state['user_input']}"
    response = llm(prompt)

    return {
        **state,
        "task_type": "summarize",
        "output": response
    }


In [ ]:
# This node handles the "translate" path.
# It asks the model to translate the input text into French.

def translate_node(state: RouterState) -> RouterState:
    prompt = f"Translate the following text to French:\n\n{state['user_input']}"
    response = llm(prompt)

    return {
        **state,
        "task_type": "translate",
        "output": response
    }


In [ ]:
# Now I’m creating a new workflow for the routing example.
# This graph will take user input, classify the task, and then run the right branch.

workflow = StateGraph(RouterState)


In [ ]:
# Adding the three main nodes for this routing workflow:
# 1) router → decides the task type
# 2) summarize → handles summaries
# 3) translate → handles French translations

workflow.add_node("router", router_node)
workflow.add_node("summarize", summarize_node)
workflow.add_node("translate", translate_node)


In [ ]:
# The workflow always starts by routing the user input,
# so the router node becomes my entry point.

workflow.set_entry_point("router")


In [ ]:
# Here I’m wiring up conditional edges.
# Based on the router's decision, the graph will either go to "summarize" or "translate".

workflow.add_conditional_edges(
    "router",
    router,
    {
        "summarize": "summarize",
        "translate": "translate",
    }
)


In [ ]:
# Both summarize and translate are valid end states for this workflow,
# so I’m marking them as finish points.

workflow.set_finish_point("summarize")
workflow.set_finish_point("translate")


In [ ]:
# Compiling the routing workflow into an executable LangGraph app.

app = workflow.compile()


In [ ]:
from IPython.display import Image, display

# Visualizing the routing graph so I can quickly confirm that
# the router branches into the two task-specific nodes as expected.

display(Image(app.get_graph().draw_png()))


In [ ]:
# First test: I’m asking the workflow to handle a translation-style request.

input_text = {
    "user_input": "Can you translate this sentence: I love programming?"
}

result = app.invoke(input_text)


In [ ]:
# Checking what the workflow produced for the first test.

print(result["output"])
print(result["task_type"])


In [ ]:
# Second test: now I’m asking for a summary instead of a translation.

input_text = {
    "user_input": (
        "Can you summarize this sentence: I love programming so much, "
        "it is the best thing ever. All I want to do is programming?"
    )
}

result = app.invoke(input_text)


In [ ]:
# And again, I’m checking the output and the detected task type.

print(result["output"])
print(result["task_type"])


In [ ]:
# Here I’m defining the state for a simple fan-out translation workflow.
# The graph will take some text, translate it into three languages in parallel,
# and then combine everything into a single output string.

class State(TypedDict):
    text: str
    french: str
    spanish: str
    japanese: str
    combined_output: str


In [ ]:
# This node handles the French translation step.

def translate_french(state: State) -> dict:
    prompt = f"Translate the following text to French:\n\n{state['text']}"
    response = llm(prompt)
    return {"french": response.strip()}


In [ ]:
# This node handles the Spanish translation step.

def translate_spanish(state: State) -> dict:
    prompt = f"Translate the following text to Spanish:\n\n{state['text']}"
    response = llm(prompt)
    return {"spanish": response.strip()}


In [ ]:
# This node handles the Japanese translation step.

def translate_japanese(state: State) -> dict:
    prompt = f"Translate the following text to Japanese:\n\n{state['text']}"
    response = llm(prompt)
    return {"japanese": response.strip()}


In [ ]:
# Once all three translations are done, this node stitches everything together
# into one combined, readable output.

def aggregator(state: State) -> dict:
    combined = f"Original Text: {state['text']}\n\n"
    combined += f"French: {state['french']}\n\n"
    combined += f"Spanish: {state['spanish']}\n\n"
    combined += f"Japanese: {state['japanese']}\n"
    return {"combined_output": combined}


In [ ]:
# Creating a new graph for the parallel translation workflow.

graph = StateGraph(State)


In [ ]:
# Adding all the translation nodes plus the aggregator to the graph.

graph.add_node("translate_french", translate_french)
graph.add_node("translate_spanish", translate_spanish)
graph.add_node("translate_japanese", translate_japanese)
graph.add_node("aggregator", aggregator)


In [ ]:
# From the START node, I’m fanning out into three parallel translation branches.

graph.add_edge(START, "translate_french")
graph.add_edge(START, "translate_spanish")
graph.add_edge(START, "translate_japanese")


In [ ]:
# Once each translation is done, all three branches feed into the aggregator node.

graph.add_edge("translate_french", "aggregator")
graph.add_edge("translate_spanish", "aggregator")
graph.add_edge("translate_japanese", "aggregator")


In [ ]:
# The aggregator is the final step before the workflow completes.

graph.add_edge("aggregator", END)


In [ ]:
# Compiling this graph into a runnable LangGraph app.
# Reusing the name 'app' here for simplicity.

app = graph.compile()


In [ ]:
# Quick test to see how the full fan-out + aggregation pipeline behaves.

input_text = {
    "text": "Good morning! I hope you have a wonderful day."
}

result = app.invoke(input_text)


In [ ]:
# Looking at the combined output with all three translations.

result
